# Яндекс Музыка

На данных Яндекс Музыки необходимо сравнить поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**План работы:**

1. Импорт данных.

2. Обзор данных.

3. Исправление критических ошибок данных.
 
4. Проверка гипотез.



## Импорт и изучение данных:

In [1]:
# Импортируем библиотеку pandas:
import pandas as pd

In [4]:
# Прочитаем файл:
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_music_project.csv')

In [5]:
# Выведем на экран первые десять строк таблицы: 
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [6]:
# Изучим общую информацию о таблице:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Вывод: 

в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков

In [11]:
# Выведем на экран названия столбцов:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [91]:
# Приведем названия в соответствие с хорошим стилем:
df=df.rename(
    columns={
    '  userID' : 'user_id',
    'Track' : 'track',
    '  City  ' : 'city',
    'Day' : 'day'
    }
)

# Проверим результат:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [15]:
# Посчитаем, сколько в таблице пропущенных значений:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

В столбцах `track` и `artist` пропуски не важны для работы. Заменим их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. 

Сделаем следующее:
* заполним и эти пропуски явными обозначениями;
* оценим, насколько они повредят расчётам. 

In [92]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = (['track'],['artist'],['genre'])
for column in columns_to_replace:
    df[column]=df[column].fillna('unknown')

    # Убедимся, что в таблице не осталось пропусков:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты:

In [18]:
# Посчитаем явные дубликаты в таблице:
df.duplicated().sum()

3826

In [93]:
# Удалим явные дубликаты:
df=df.drop_duplicates()

# Проверим на отсутствие дубликатов:
df.duplicated().sum()

0

Работа с неявными дубликатами:

In [21]:
# Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Есть следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

In [94]:
# Заменим неявные дубликаты:
df['genre']=df['genre'].replace('hip', 'hiphop')
df['genre']=df['genre'].replace('hop', 'hiphop')
df['genre']=df['genre'].replace('hip-hop', 'hiphop')

# Проверим замену:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Я исправила заголовки, чтобы упростить работу с таблицей. 

Без дубликатов исследование станет более точным.

Пропущенные значения заменила на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе.


In [25]:
# Сгруппируем данные по городу и посчитаем прослушивания в каждой группе:
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

Вывод: В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.


In [26]:
# Сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу:
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

Вывод: В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [35]:
# Напишем функцию, которая объединит эти два расчёта: 
def number_tracks(day,city):
    track_list=df.loc[df['day'] == day].loc[df['city']==city]
    track_list_count=track_list['user_id'].count()
    return track_list_count

Получим данные для каждого города в каждый из трёх дней:

In [65]:
print('Количество прослушиваний в Москве по понедельникам =', (number_tracks('Monday','Moscow')))
print('Количество прослушиваний в Санкт-Петербурге по понедельникам =', (number_tracks('Monday','Saint-Petersburg')))
print('Kоличество прослушиваний в Москве по средам =', (number_tracks('Wednesday','Moscow')))
print('Количество прослушиваний в Санкт-Петербурге по средам =', (number_tracks('Wednesday','Saint-Petersburg')))
print('Количество прослушиваний в Москве по пятницам =', (number_tracks('Friday','Moscow')))
print('Количество прослушиваний в Санкт-Петербурге по пятницам =', (number_tracks('Friday','Saint-Petersburg')))

Количество прослушиваний в Москве по понедельникам = 15740
Количество прослушиваний в Санкт-Петербурге по понедельникам = 5614
Kоличество прослушиваний в Москве по средам = 11056
Количество прослушиваний в Санкт-Петербурге по средам = 7003
Количество прослушиваний в Москве по пятницам = 15945
Количество прослушиваний в Санкт-Петербурге по пятницам = 5895


Создадим таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`.

In [42]:
results=pd.DataFrame(data=[
['Москва', 15740, 11056, 15945], ['Санкт-Петербург', 5614, 7003, 5895]
], 
columns=['city', 'monday', 'wednesday', 'friday'])
print(results)

              city  monday  wednesday  friday
0           Москва   15740      11056   15945
1  Санкт-Петербург    5614       7003    5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [43]:
# Сохраним таблицы с данными в две переменные:
moscow_general=df[df['city']=='Moscow']
spb_general=df[df['city']=='Saint-Petersburg']

Создадим функцию, которая возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени:

In [48]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10) 

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [79]:
print('Утро понедельника в Москве:', genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))
print()
print('Утро понедельника в Петербурге:', genre_weekday(spb_general, 'Monday', '07:00', '11:00'))
print()
print('Вечер пятницы в Москве:', genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))
print()
print('Вечер пятницы в Петербурге:', genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

Утро понедельника в Москве: genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre, dtype: int64

Утро понедельника в Петербурге: genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

Вечер пятницы в Москве: genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

Вечер пятницы в Петербурге: genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` и `spb_general` по жанру и посчитаем прослушивания треков каждого жанра:

Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres` и `spb_general` соответственно:

In [86]:
moscow_genres=moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres=spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [90]:
# Выведем на экран первые десять строк `moscow_genres`: 
print('Популярные жанры в Москве:', moscow_genres.head(10))
print()
# Выведем на экран первые десять строк `spb_genres`: 
print('Популярные жанры в Питере:', spb_genres.head(10))

Популярные жанры в Москве: genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Популярные жанры в Питере: genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Я проверила три гипотезы и установила:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.